# Image Classification with PyTorch: Transfer learning 

[Pablo Carballeira] (http://dymas.ii.uam.es/webvpu/gti/user/186/), Escuela Politecnica Superior, Universidad Autónoma de Madrid.

Parts of this code have been adapted from then work of Kevin McGuinness (http://www.eeng.dcu.ie/~mcguinne/), School of Electronic Engineering, Dublin City University, and the work of Ben Trevett (https://github.com/bentrevett), Heriot-Watt University

You can find documentation about working in Colab here (https://colab.research.google.com/notebooks/intro.ipynb)

---

This lab will investigate the use of transfer learning for a simple binary classification problem. We will work with the "Alien vs Predator" dataset that that originally comes from a [Kaggle competition](https://www.kaggle.com/pmigdal/alien-vs-predator-images). The task is to classify images as containing either the Alien creature from the Ridley Scott movie, or the Predator creature from the 1987 action classic Predator starring Arnold Schwarzenegger.

![AVP](https://crowdcast-prod.imgix.net/-Kab5fihhH1Q3R92MsOJ/event-cover-4763?w=800)


The dataset contains around 700 training examples and 200 validation examples, which are evenly balanced between the classes. However, 700 examples is quite a small dataset when comes to deep learning, and training a neural network from scratch (from a random initialization) would **overfit**.

This notebook will look at simple transfer learning techniques:
- Using a pretrained network **"off-the-shelf"** as a feature extractor. 
- **Fine tuning** a pretrained network for the new task.

In addition to the accuracy results, we will look at the t-SNE representations of the trained from scratch, pre-trained and fine-tuned features to understand their capacity to separate both classes



# Instructions

Anywhere you see a **???** in the code below, fill in in with the correct code.

# Import packages


Find the PyTorch docs at https://pytorch.org/docs/stable/index.html 

Tutorials: https://pytorch.org/tutorials/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn import decomposition
from sklearn import manifold

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torchvision.datasets.utils import download_file_from_google_drive
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import copy
import random
import time

# Enable GPU acceleration

Open to the Edit menu and select *Notebook settings* and check that *GPU* is selected under hardware accelerator.


In [ ]:
# make sure to enable GPU acceleration!
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


We set the random seed so all of our experiments can be reproduced.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Dataset download

Here we will use the "Alien vs Predator" dataset, which was originally taken from a [Kaggle competition](https://www.kaggle.com/pmigdal/alien-vs-predator-images). You can download it using this [Google Drive link](https://drive.google.com/file/d/1MFC6nE5XOdeP7xiz1XkCTJwNvHPHG524/view?usp=sharing). Here I use wget to download it directly from Google Drive.

In [ ]:
!mkdir data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MFC6nE5XOdeP7xiz1XkCTJwNvHPHG524' -O data/avp.tar


0it [00:00, ?it/s]

Extract the data and remove the tar file

In [ ]:
#!unzip data/avp.zip -d data
#!rm data/avp.zip

!cd data && tar xf avp.tar
!rm data/avp.tar

!ls data/avp

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors
ls: cannot access 'data/avp': No such file or directory


# Create the dataset

We will use the `ImageFolder` class to create the dataset. This works whenever we have a folder with subfolders, each of which represents a class. You can explore the `./content/data/avp` folder to check the structure of directories


First, let's take a look at some examples from the training set, to see how these images look like

In [ ]:
visual_train_set = ImageFolder('data/avp/train')
for i in range(5):
    plt.figure()
    plt.imshow(visual_train_set[i][0])
    plt.title(visual_train_set.classes[visual_train_set[i][1]])
    plt.axis('off')
    plt.show()

    plt.figure()
    plt.imshow(visual_train_set[i+400][0])
    plt.title(visual_train_set.classes[visual_train_set[i+400][1]])
    plt.axis('off')
    plt.show()

FileNotFoundError: ignored

We can see that this dataset is composed by images with different sizes and aspect ratios. We have to take this into account when creating the training and validation samples, as CNNs are usually designed for square images of fixed-size.

# Define the transformations

Here, we specify the set of transforms to be applied to the data. In addition to normalization, we apply image resizing and cropping to adapt the sizes of the images to the input of the CNN. For the time being, we are going to:
* resize the images by resizing the smallest dimension to 192 pixels (maintaining the aspect ratio), and
* take a  192x192 crop in from the image center.

Later, we will see more efficient ways to do this image cropping in the training and validation data. 

In [ ]:
# Resize image and take a center crop. Check torchvision help to find the correct functions
train_transform = transforms.Compose([
    ???
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))                  
])

# apply, the same transformation to the validation data
valid_transform = ???

# Create training and validation sets using ImageFolder, and applying the 
# transformation defined above
train_set = ??? 
valid_set = ???

# Create training and validation loaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)

In [ ]:
print(f'train samples: {len(train_set)}  validation samples: {len(valid_set)}')

# Training from scratch

First of all, let's see what happens if we train a CNN from scratch using the Alien vs Predator dataset.

Let's use the ResNet-50 model described by Kaiming He in his seminal paper [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385). We will load the model, with random weights, and take a look at its layers

## Load the model and modify last FC layer

In [ ]:
# create a resnet50 model as before but do NOT load pre-trained weights
model = models.resnet50(pretrained=False)
model.to(device)

This model architecture is defined to be trained in the ImageNet dataset (as in the vast majority of cases). Therefore, the last FC layer is set to output scores for the 1000 classes of the ImageNet dataset. The first thing we need to do is to change the last FC layer so it outputs the scores of our two classes (Alien and Predator)

In [ ]:
# Replace the fc layer of the model with a FC layer of appropriate size
model.fc = ???

model.to(device)


## Training the model

Let's define the training functions ...

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
def save_checkpoint(optimizer, model, epoch, filename):
    checkpoint_dict = {
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint_dict, filename)


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [ ]:
!mkdir -p checkpoints

In [ ]:
def train_for_epoch(optimizer):

    # put the model in training mode
    model.train()

    train_losses = []

    for batch, target in train_loader:

        # data to GPU
        batch = batch.to(device)
        target = target.to(device)

        # reset optimizer
        optimizer.zero_grad()

        # forward pass
        predictions = model(batch)

        # calculate loss
        loss = criterion(predictions, target)

        # backward pass
        loss.backward()

        # parameter update
        optimizer.step()

        # track loss
        train_losses.append(float(loss.item()))

    train_losses = np.array(train_losses)
    train_loss = np.mean(train_losses)  

    return train_loss


def validate():
    model.eval()

    valid_losses = []
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch, target in valid_loader:

            # move data to the device
            batch = batch.to(device)
            target = target.to(device)

            # make predictions
            predictions = model(batch)

            # calculate loss
            loss = criterion(predictions, target)

            # track losses and predictions
            valid_losses.append(float(loss.item()))
            y_true.extend(target.cpu().numpy())
            y_pred.extend(predictions.argmax(dim=1).cpu().numpy())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    valid_losses = np.array(valid_losses)

    # calculate validation accuracy from y_true and y_pred
    valid_accuracy = metrics.accuracy_score(y_true, y_pred)

    # calculate the mean validation loss
    valid_loss = np.mean(valid_losses)

    return valid_loss, valid_accuracy


# training loop
def train(epochs, optimizer, first_epoch=1):

    train_losses, valid_losses = [],  []

    for epoch in range(first_epoch, epochs+first_epoch):

        # train
        train_loss = train_for_epoch(optimizer)

        # validation
        valid_loss, valid_acc = validate()

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
    
        # print status
        print(f'[{epoch:02d}] train loss: {train_loss.mean():0.04f}  '
              f'valid loss: {valid_loss:0.04f}  '
              f'acc: {valid_acc:0.04f}')
        
        # save checkpoint
        save_checkpoint(optimizer, model, epoch, f'checkpoints/avp_{epoch:03d}.pkl')
        #torch.save(model, f'checkpoints/avp_{epoch:03d}.pkl')

    return train_losses, valid_losses


... and train the network for 15 epochs

In [ ]:
# Create an SGD optimizer with learning rate 0.01 and momentum 0.9
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Now train the model using the above optimizer for 15 epochs
train_losses, valid_losses = train(15, optimizer)

## Plot the learning curves




In [ ]:
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(10,6))
plt.plot(epochs, train_losses, '-o', label='Training loss')
plt.plot(epochs, valid_losses, '-o', label='Validation loss')

plt.legend()
plt.title('Learning curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(epochs)
plt.ylim(0,5)
plt.show()

We obtain an ~75% accuracy, which is not bad, but we will see that we can do much better using transfer learning

## Features trained from scratch: t-SNE representation

Let's take a look at the t-SNE representations of the features of the model. The features are obtained just before the FC layer (remember that the ResNet models include a Global Average Pooling layer followed by a single FC layer)

We want to check their representation capacity, in order to correctly separate both classes, and we will compare these features trained from scratch with the pre-trained and fine-tuned features. 

First, we define the functions for the t-SNE representation

In [ ]:
def get_representations(model, iterator, device):

    model.eval()

    outputs = []
    intermediates = []
    labels = []

    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)

            y_pred = model(x)

            outputs.append(y_pred.cpu())
            labels.append(y)
        
    outputs = torch.cat(outputs, dim = 0)
    labels = torch.cat(labels, dim = 0)

    return outputs, labels

In [ ]:
def plot_representations(data, labels, classes, n_images = None):
    
    if n_images is not None:
        data = data[:n_images]
        labels = labels[:n_images]
        
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(111)
    scatter = ax.scatter(data[:, 0], data[:, 1], c = labels, cmap = 'tab10')
    handles, labels = scatter.legend_elements()
    legend = ax.legend(handles = handles, labels = classes)

In [ ]:
def get_tsne(data, n_components = 2, n_images = None):
    
    if n_images is not None:
        data = data[:n_images]
        
    tsne = manifold.TSNE(n_components = n_components, random_state = 0)
    tsne_data = tsne.fit_transform(data)
    return tsne_data

Get the features and labels of the samples of the validation set.

The first thing we need to do is to remove the last fully connected layer. An empty `nn.Sequential` layer can be used as an Identity mapping. 



In [ ]:
model.fc = ???

Now obtain the features and labels for the validation samples using the functions defined above

In [ ]:
valid_features, valid_labels = ???

Plot the t-SNE representations

In [ ]:
N_IMAGES = 200

output_tsne_data = get_tsne(valid_features, n_images = N_IMAGES)
plot_representations(output_tsne_data, valid_labels, valid_set.classes, n_images = N_IMAGES)

# Load a pre-trained ResNet model

Now let's load a pretrained ResNet50 model and try some transfer learning approaches


In [ ]:
model = models.resnet50(pretrained=True)
model.to(device);

# Off-the-shelf transfer learning

Let's start with "off-the-shelf" transfer learning. Here we will just use the pretrained model as a feature extractor and then train a linear classifier on top of these pretrained features.



## Pre-trained features: t-SNE representation

In [ ]:
# use the code above to extract the pre-trained features (before FC layer) and plot
# its t-SNE representation in two dimensions. We recommend that you encapsulate the code 
# in a single cell. You will reuse it later
???

## Pre-trained features for training and validation data

In [ ]:
# extraction of training and validation features and labels
train_features, train_labels = ??? 
valid_features, valid_labels = ??? 

print(f'train features are {train_features.shape}')
print(f'valid features are {valid_features.shape}')

## Train a Support Vector Machine on top of the pre-trained features

Now, train a classical machine learning model on the representations. Let's try a **support vector machine** with a linear kernel as the classifier.

In [ ]:
from sklearn.svm import SVC

In [ ]:
# define a linear SMV classifier using SVC, and use the fit function to train it using the training features
classifier = ???
???

In [ ]:
# make predictions for the validation feature using the classifier trained above
y_hat = ???

In [ ]:
accuracy = metrics.accuracy_score(valid_labels, y_hat)
print(f'Accuracy: {accuracy:0.3f}')

You should get 90.5% accuracy off the shelf: not bad! The validation set is balanced, so this is an excellent start.

In [ ]:
print(metrics.classification_report(valid_labels, y_hat, target_names=train_set.classes))

# Fine tuning

Let's try replace the last layer of the ResNet and fine tune it on the AVP dataset for a few epochs

## Modify the model

In [ ]:
model = models.resnet50(pretrained=True)
# Replace fc with an fc layer of appropuate size (same as in training from scratch)
model.fc = ???

model.to(device);

## Train (fine-tune) the model

Train the final layer for a few epochs to get it to a good place (remember, it starts off with random weights but the rest of the network has pre-trained weights from ImageNet)

In [ ]:
final_layer_optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
train_losses, valid_losses = train(2, final_layer_optimizer)

Now fine tune whole network with a lower learning rate

In [ ]:
full_network_optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# train the model using the full network optimizer above for 10 epochs starting at epoch 3
train_losses_full, valid_losses_full =  ???

train_losses.extend(train_losses_full)
valid_losses.extend(valid_losses_full)

In [ ]:
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(10,6))
plt.plot(epochs, train_losses, '-o', label='Training loss')
plt.plot(epochs, valid_losses, '-o', label='Validation loss')

plt.legend()
plt.title('Learning curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(epochs)
plt.show()

Use the load_checkpoint function to load the model with the best performance

In [ ]:
???

## Fine-tuned features: t-SNE representations 

In [ ]:
# use the code above to extract the fine-tuned features (before FC layer) and plot
# its t-SNE representation in two dimensions
???

# Data augmentation

Let's see if we can improve the performance of the fine-tuned model by applying some data augmentation to the training data. 

## Data augmentation transforms

Now we will define a set of transforms that will be applied to the data that implement basic data augmentation techniques: Random Cropping and Random Horizontal Flipping. 

The train transform is stochastic, which will help prevent overfitting when we fine-tune the model. The validation transform is deterministic, which ensures we will have consistent results on the same data.

In [ ]:
# define a transformation for the training data that resizes the image to 224x224 pixels,
# extracts a random 192x192 crop, and applies horizontal flip with 0.5 probability. 
# Use the same data normalization we have used above
train_transform = ???


# define a similar transformation for the validation data that resizes the image to 224x224 pixels,
# but now extracts a 192x192 crop from the image center
valid_transform = ???

# Create training and validation sets using ImageFolder, and applying the 
# transformation defined above
train_set = ??? 
valid_set = ???

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
# Create a loader for the validation set, but disable shuffling of the validation data
# (it is unnecessary to shuffle validation data)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)

In [ ]:
print(f'train samples: {len(train_set)}  validation samples: {len(valid_set)}')

## Fine tuning

Use the previous code to fine-tune the pre-trained ResNet model, using the augmented training data, plot the learning graphs, and get the t-SNE representations of the features of the last layer

In [ ]:
???